In [1]:
import os
# from datetime import datetime
import datetime
from tkinter import Tk,filedialog
from tkinter import *
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout
from CoastSeg import zoo_model_module
from IPython.display import  display

from doodleverse_utils.prediction_imports import do_seg
from doodleverse_utils.imports import simple_resunet, simple_unet, simple_satunet, custom_resunet, custom_unet, mean_iou, dice_coef
import tensorflow as tf

from CoastSeg import file_functions

#suppress tensorflow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
root = Tk()
out = widgets.Output()

output_select_all = widgets.Output()

Version:  2.9.1
Eager mode:  True
Version:  2.9.1
Eager mode:  True
GPU name:  []
Num GPUs Available:  0


In [2]:
# Date Widgets
start_date=widgets.DatePicker(
    description='Start Date',
    value=datetime.date(2018, 12, 1),
    disabled=False,
)
end_date=widgets.DatePicker(
    description='End Date',
    value=datetime.date(2019, 1, 1),
    disabled=False,
)
date_instr=widgets.HTML(
    value="<b>Pick a date:<b>",
    layout=Layout(padding='10px')
)
dates_box=widgets.HBox([start_date,end_date])
dates_vbox=widgets.VBox([date_instr,dates_box])


collection_instr=widgets.HTML(
    value="<b>Pick a collection<b>\
        <br> C01: LandSat Collection 1 \
        <br> C02: LandSat Collection 2  [2022/01/01 and beyond] \
        <br>      - Landsat 9 is only available in C02",
    layout=Layout(padding='10px')
)
collection_radio=widgets.RadioButtons(
    options=['C01', 'C02'],
    value='C01',
    description='Collection:',
    disabled=False
)
collection_vbox=widgets.VBox([collection_instr,collection_radio])

controls_vbox=widgets.VBox([dates_vbox,collection_vbox])
controls_vbox

In [3]:
data_selection=widgets.Dropdown(
    options=[('Select Your Images', 1), ('Use Data Folder', 2)],
    value=1,
    description='Number:',
)

use_GPU_output = widgets.Output()
GPU_checkbox=widgets.Checkbox(
    value=False,
    description='Use GPU?',
    disabled=False,
    indent=False
)


In [5]:
#Rerun this to save the selections
display(data_selection, GPU_checkbox, use_GPU_output,controls_vbox)

Dropdown(description='Number:', options=(('Select Your Images', 1), ('Use Data Folder', 2)), value=1)

Checkbox(value=False, description='Use GPU?', indent=False)

Output()

In [ ]:
def get_jpgs_from_data():
    """Copies all the jpgs from the data folder in CoastSeg to a new folder, where the model
    will save the computed segmentations."""
    # Data folder location 
    src_path=os.getcwd()+os.sep+"data"
    if os.path.exists(src_path):
        file_functions.rename_jpgs(src_path)
        # Create a new folder to hold all the data
        location=os.getcwd()
        name="segmentation_data"
        new_folder=file_functions.mk_new_dir(name,location)
        glob_str = src_path + str(os.sep + "**" + os.sep) * 3 + "*jpg"
        file_functions.copy_files_to_dst(src_path,new_folder, glob_str)
        return new_folder
    else:
        print("ERROR: Cannot find the data directory in CoastSeg")
        raise Exception("ERROR: Cannot find the data directory in CoastSeg")


In [ ]:
if data_selection.value == 1:
    # Prompt the user to select a directory of images                              
    root.withdraw()                                        # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True)   # Raise the root to the top of all windows.
    root.filename =  filedialog.askdirectory(initialdir = "/samples",title = "Select directory of images (or npzs) to segment")
    # Save the filename as an attribute of the button
    if root.filename:
        sample_direc= root.filename
        print(f"{sample_direc} will be segmented")
    else:
        raise Exception("You must select a valid directory first!")
elif data_selection.value ==2:
    # Use the data folder as the input for segmentation
    print("Loading in the jpgs from the data directory")
    # Copy the jpgs from data to a new folder called segmentation_data_[datetime]
    sample_direc = get_jpgs_from_data()
    print(f"Contents of the data directory saved in {sample_direc}")

In [ ]:
if GPU_checkbox.value == False:
    print("Not using the GPU")
    ## to use the CPU (not recommended):
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
else:
    print("Using the GPU")
    ##use the first available GPU
    os.environ['CUDA_VISIBLE_DEVICES'] = '0' #'1'

In [ ]:
dataset='RGB'
dataset_id ='landsat_6229071'
model_choice='ENSEMBLE'
zoo_model=zoo_model_module.Zoo_Model()
# # First download the specified model
zoo_model.download_model(dataset,dataset_id)
# # Get weights as list
Ww=zoo_model.get_weights_list(model_choice)
# Load the model from the config files
model, model_list, config_files, model_types=zoo_model.get_model(Ww)
metadatadict=zoo_model.get_metadatadict(Ww,config_files, model_types)
# # Compute the segmentation
zoo_model.compute_segmentation(sample_direc, model_list, metadatadict)
